In [1]:
from dotenv import load_dotenv
import time
import sys

import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import os
import json
import pandas as pd
load_dotenv()
PGHOST=os.environ.get("PGHOST")
PGDB=os.environ.get("PGDB")
PGUSR=os.environ.get("PGUSR")
PGPWD=os.environ.get("PGPWD")
PGPORT=os.environ.get("PGPORT")

In [ ]:
""" create a class that will receive the connection parameters"""   
class Database(object):
    def __init__(self, host, db, usr, pwd, port, drivername, **kwargs):
        self.host = host
        self.db = db
        self.usr = usr
        self.pwd = pwd
        self.port = port
        self.driver = drivername
        self.kwargs = kwargs
        self.url= sqlalchemy.url.URL(self.driver, self.host, self.port, self.db, self.usr, self.pwd, **self.kwargs)
    

In [5]:
"""" create a function thal will return all tables of a Database class for a postgres database sqlalchemy"""
def get_tables_postgres(db):
    conn = sqlalchemy.create_engine(f'{db.driver}://{db.usr}:{db.pwd}@{db.host}:{db.port}/{db.db}')
    tables = pd.read_sql_query("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES", conn)
    return tables

"""" create a function thal will return all columns of a Database class for a postgres database sqlalchemy"""
def get_columns_postgres(db):
    conn = sqlalchemy.create_engine(f'{db.driver}://{db.usr}:{db.pwd}@{db.host}:{db.port}/{db.db}')
    columns = pd.read_sql_query("SELECT TABLE_NAME, COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS", conn)
    return columns
    


'" create a function thal will return all tables of a Database class'

In [ ]:
"""create a function that will return all tables of a Database class for a mssql database"""
def get_tables_mssql(db):
    conn = sqlalchemy.create_engine(f'{db.driver}://{db.usr}:{db.pwd}@{db.host}:{db.port}/{db.db}')
    tables = pd.read_sql_query("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES", conn)
    return tables
"""create a function that will return all columns of a Database class for a mssql database"""
def get_columns_mssql(db):
    conn = sqlalchemy.create_engine(f'{db.driver}://{db.usr}:{db.pwd}@{db.host}:{db.port}/{db.db}')
    columns = pd.read_sql_query("SELECT TABLE_NAME, COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS", conn)
    return columns

In [ ]:
"""create a function that will return all tables of a Database class for a mysql database"""
def get_tables_mysql(db):
    conn = sqlalchemy.create_engine(f'{db.driver}://{db.usr}:{db.pwd}@{db.host}:{db.port}/{db.db}')
    tables = pd.read_sql_query("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES", conn)
    return tables
"""create a function that will return all columns of a Database class for a mysql database"""
def get_columns_mysql(db):
    conn = sqlalchemy.create_engine(f'{db.driver}://{db.usr}:{db.pwd}@{db.host}:{db.port}/{db.db}')
    columns = pd.read_sql_query("SELECT TABLE_NAME, COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS", conn)
    return columns
    


In [ ]:
"""create a endpoint that will receive a database class and return all tables and all columns of a given database"""
def get_tables_columns(db):
    if db.driver == 'postgresql+psycopg2':
        tables = get_tables_postgres(db)
        columns = get_columns_postgres(db)
    elif db.driver == 'mssql+pyodbc':
        tables = get_tables_mssql(db)
        columns = get_columns_mssql(db)
    elif db.driver == 'mysql':
        tables = get_tables_mysql(db)
        columns = get_columns_mysql(db)
    else:
        print('driver not supported')
        sys.exit()
    return tables, columns

In [7]:
"""create a class that will receive two columns, then  format a insert instruction from the second column to the first column"""
class Insert(object):
    def __init__(self, table, columns):
        self.table = table
        self.columns = columns
    def insert(self):
        return f"INSERT INTO {self.table} ({', '.join(self.columns)}) VALUES ({', '.join(['%s' for _ in self.columns])})"



In [9]:
"""create a class that will receive a table and columns, then format a select instruction from the columns to the table"""
class Select(object):
    def __init__(self, table, columns):
        self.table = table
        self.columns = columns
    def select(self):
        return f"SELECT {', '.join(self.columns)} FROM {self.table}"
        


In [10]:
"""create a class thal will receive a database migration project whith project id, destination database, source database, and a list of tables and columns to migrate
then it will create a new database class with the destination database and a list of tables and columns to migrate"""
class Migrate(object):
    def __init__(self, project_id, destination_db, source_db):
        self.project_id = project_id
        self.destination_db = destination_db
        self.source_db = source_db
        self.destination_db = Database(destination_db.host, destination_db.db, destination_db.usr, destination_db.pwd, destination_db.port, destination_db.driver, **destination_db.kwargs)
        self.source_db = Database(source_db.host, source_db.db, source_db.usr, source_db.pwd, source_db.port, source_db.driver, **source_db.kwargs)
        self.source_tables, self.source_columns = get_tables_columns(self.source_db)
        self.destination_tables, self.destination_columns = get_tables_columns(self.destination_db)


"""create a function that will receive a Migrate class and select the data from each source table and columns and insert it to the destination database"""
def migrate(migrate):
    for table in migrate.source_tables['TABLE_NAME']:
        if table in migrate.destination_tables['TABLE_NAME']:
            print(f'{table} already exists in destination database')
        else:
            print(f'creating {table} table in destination database')
            migrate.destination_db.engine.execute(f'CREATE TABLE {table} ({", ".join(migrate.source_columns[migrate.source_columns["TABLE_NAME"] == table]["COLUMN_NAME"])})')
        print(f'inserting data from {table} table to destination database')
        migrate.destination_db.engine.execute(Insert(table, migrate.source_columns[migrate.source_columns["TABLE_NAME"] == table]["COLUMN_NAME"]).insert(), migrate.source_db.engine.execute(Select(table, migrate.source_columns[migrate.source_columns["TABLE_NAME"] == table]["COLUMN_NAME"]).select()).fetchall())
    print('migration completed')


In [ ]:
"""create """

In [ ]:
sqlalchemy.